In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict

## Reading data area

In [74]:
# Read data for pivot table maipulation
replace_uae_en = pd.read_csv('UAE-EN-[flydubai]-Auction insight.csv', encoding='utf-16le', skiprows=2, sep='\t')
replace_ksa_en = pd.read_csv('KSA-EN-[flydubai]-Auction insight.csv', encoding='utf-16le', skiprows=2, sep='\t')
replace_uae_ar = pd.read_csv('UAE-AR-[flydubai].csv', encoding='utf-16le', skiprows=2, sep='\t')
replace_ksa_ar = pd.read_csv('KSA-AR-[flydubai].csv', encoding='utf-16le', skiprows=2, sep='\t')

In [75]:
# Read data for dropping cols
drop_uae_en = pd.read_csv('UAE-EN-[fly dubai-Auction insight.csv', encoding='utf-16le', skiprows=2, sep='\t')
drop_uae_en_terms = pd.read_csv('UAE-EN-[flydubai] + terms -Auction insights.csv', encoding='utf-16le', skiprows=2, sep='\t')
drop_ksa_en = pd.read_csv('KSA-EN-[fly dubai]-Auction insight.csv', encoding='utf-16le', skiprows=2, sep='\t')
drop_ksa_en_terms = pd.read_csv('KSA-EN-[flydubai]+terms-Auction insight.csv', encoding='utf-16le', skiprows=2, sep='\t')

In [76]:
names_dict = defaultdict()

## Give Data frames a name attribute

In [77]:
replace_uae_en.name = 'replace_uae_en'
replace_ksa_ar.name = 'replace_ksa_ar'
replace_ksa_en.name = 'replace_ksa_en'
replace_uae_ar.name = 'replace_uae_ar'

## Make methods to accept any args, make floats and make pivots

In [78]:
def replace_dunder(*args):
    for df in args:
        df['Position above rate'].replace(to_replace=' --', value='0.0', inplace=True)
        df['Outranking share'].replace(to_replace=' --', value='0.0', inplace=True)

def strip_and_float(*args):
    for df in args:
        df['Impression share'] = [float(val.strip('% | <')) for val in df['Impression share']]
        df['Position above rate'] = [float(val.strip('% | < |')) for val in df['Position above rate']]
        df['Outranking share'] = [float(val.strip('% | < |')) for val in df['Outranking share']]

def make_pivots(*args):
    for df in args:
        names_dict['pivot'+df.name] = pd.pivot_table(data=df, index=['Display URL domain', 'Device'],
               values=['Impression share', 'Position above rate', 'Outranking share'])
        
        names_dict['pivot'+df.name] = names_dict['pivot'+df.name].astype(str) + '%'
        
def clean_dfs(*args):
    replace_dunder(*args)
    strip_and_float(*args)
    make_pivots(*args)

## Drop some cols from datasets

In [79]:
def drop_cols(*args):
    for df in args:
        df.drop(labels=['Avg. position', 'Overlap rate', 'Top of page rate', 'Abs. Top of page rate'], axis=1, inplace=True)

In [80]:
alldfs = [var for var in dir() if isinstance(eval(var), pd.core.frame.DataFrame)]
for df in [replace_ksa_ar, replace_ksa_en, replace_uae_ar, replace_uae_en]:
    clean_dfs(df)

In [82]:
drop_cols(drop_ksa_en, drop_ksa_en_terms, drop_uae_en, drop_uae_en_terms)

## Write to excel sheets with multiple pd.DataFrames on same sheet

In [84]:
writer = pd.ExcelWriter('auction_insights.xlsx')

In [85]:
names_dict['pivotreplace_ksa_ar'].to_excel(writer, sheet_name='KSA-AR', startrow=0, startcol=0)
names_dict['pivotreplace_ksa_en'].to_excel(writer, sheet_name='KSA-EN', startrow=0, startcol=0)
names_dict['pivotreplace_uae_ar'].to_excel(writer, sheet_name='UAE-AR', startrow=0, startcol=0)
names_dict['pivotreplace_uae_en'].to_excel(writer, sheet_name='UAE-EN', startrow=0, startcol=0)

drop_ksa_en.to_excel(writer, sheet_name='KSA-EN', startrow=20, startcol=0, index=False)
drop_uae_en.to_excel(writer, sheet_name='UAE-EN', startrow=20, startcol=0, index=False)

drop_ksa_en_terms.to_excel(writer, sheet_name='KSA-EN', startrow=30, startcol=0, index=False)
drop_uae_en_terms.to_excel(writer, sheet_name='UAE-EN', startrow=30, startcol=0, index=False)

In [86]:
writer.save()